## 气象数据插值


插值就是在离散数据的基础上补插连续函数。利用插值可通过函数在有限个点处的取值状况，估算出函数在其它点处的近似值。这里我们主要从时间插值和空间插值两个方面来说。

空间插值：在同一个时间维度下，对具有空间属性的气象数据插值到特定的空间位置上。

时间插值：根据现有数据对未来数据进行插值，称为外推；根据现有数据对时间范围内的某个时间点进行插值，称为内插，今天主要谈的是内插。

## 空间插值

### 从站点到网格

将站点数据插值为网格数据(给定站点数据的经纬度和变量值，插值为规则经纬度网格的值)

插值时特别需要注意网格分辨率的大小，过密的网格可能会导致插值结果出现异常值；过稀的网格可能会导致最后结果遗漏某些较小的高低值中心。需要根据具体站点数据和插值结果对格点分辨率进行调整。


In [8]:
import pandas as pd
import numpy as np
f = pd.read_csv('/home/mw/input/moyu1828/1957_2012_tm_year_avg.txt',sep='\s+',names=['sta','lat','lon','alt','year','tm'],na_values=-99.90)
data = f.loc[f.year==2006]
data = data.dropna(axis=0,how='any')

lat = data.lat.values/100
lon = data.lon.values/100
tm = data.tm.values
sta = data.sta.values

print(data.agg(['min', 'max']))

       sta   lat    lon    alt  year     tm
min  50136  1653   7523     20  2006  -4.65
max  59985  5297  13297  48000  2006  27.19


#### Cressman插值

Cressman插值法是一种反距离权重插值法，它使用网格点与在一定范围内所有站点的距离反比作为权重，并以此计算网格点值的插值模型。

metpy

In [14]:
from metpy.interpolate import inverse_distance_to_grid

lon_grid = np.arange(75.0, 134.0, 1.0) 
lat_grid = np.arange(16.0, 54.0, 1.0) 

lon_gridmesh, lat_gridmesh = np.meshgrid(lon_grid, lat_grid)

tm_grid = inverse_distance_to_grid(lon,lat,tm,lon_gridmesh,lat_gridmesh,r=15,min_neighbors=3)

In [17]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker

fig = plt.figure(figsize=(12,8))
ax1 = fig.add_axes([0.1, 0.1, 0.8, 0.4],projection = ccrs.PlateCarree(central_longitude=90))
leftlon, rightlon, lowerlat, upperlat = (0,180,0,90)
img_extent = [leftlon, rightlon, lowerlat, upperlat]
ax1.set_extent(img_extent, crs=ccrs.PlateCarree())
# ax1.add_feature(cfeature.COASTLINE.with_scale('50m')) 
# ax1.add_feature(cfeature.LAKES, alpha=0.5)
ax1.set_xticks(np.arange(60,150+30,30), crs=ccrs.PlateCarree())
ax1.set_yticks(np.arange(0,60+30,30), crs=ccrs.PlateCarree())
lon_formatter = cticker.LongitudeFormatter()
lat_formatter = cticker.LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)
c1 = ax1.scatter(lon,lat,s=2, zorder=0,c=tm,transform=ccrs.PlateCarree(), cmap=plt.cm.Reds)

position=fig.add_axes([0.35, 0.02,  0.35, 0.025])
fig.colorbar(c1,cax=position,orientation='horizontal',format='%d',)

<Figure size 864x576 with 2 Axes>

In [18]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker

fig = plt.figure(figsize=(12,8))
ax1 = fig.add_axes([0.1, 0.1, 0.8, 0.4],projection = ccrs.PlateCarree(central_longitude=90))
leftlon, rightlon, lowerlat, upperlat = (0,180,0,90)
img_extent = [leftlon, rightlon, lowerlat, upperlat]
ax1.set_extent(img_extent, crs=ccrs.PlateCarree())
# ax1.add_feature(cfeature.COASTLINE.with_scale('50m')) 
# ax1.add_feature(cfeature.LAKES, alpha=0.5)
ax1.set_xticks(np.arange(60,150+30,30), crs=ccrs.PlateCarree())
ax1.set_yticks(np.arange(0,60+30,30), crs=ccrs.PlateCarree())
lon_formatter = cticker.LongitudeFormatter()
lat_formatter = cticker.LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)
c1 = ax1.contourf(lon_grid,lat_grid,tm_grid, zorder=0,#levels =np.arange(-10,10.5,0.5) , 
                     extend = 'both', transform=ccrs.PlateCarree(), cmap=plt.cm.Reds)

position=fig.add_axes([0.35, 0.02,  0.35, 0.025])
fig.colorbar(c1,cax=position,orientation='horizontal',format='%d',)

<Figure size 864x576 with 2 Axes>

### 克里金插值

conda install -c conda-forge pykrige

一种对已知样本加权平均以估计平面上的未知点，并使得估计值与真实值的数学期望相同且方差最小的地统计学过程

In [ ]:
from pykrige.ok import OrdinaryKriging

krige = OrdinaryKriging(lon,lat,tm,variogram_model="linear",verbose=False,enable_plotting=False)
lon_grid = np.arange(75.0, 134.0, 1.0) 
lat_grid = np.arange(16.0, 54.0, 1.0) 
tm_grid,ss = krige.execute("grid", lon_grid, lat_grid)


### 从网格到站点

使用Xarray从网格数据，插值到站点数据。

In [22]:
print(lon_grid.shape,lat_grid.shape,tm_grid.shape)
print(lon.shape,lat.shape,tm.shape)

import xarray as xr
tm_da = xr.DataArray(tm_grid, coords=[lat_grid, lon_grid], dims=['lat', 'lon'])
lat = xr.DataArray(lat, coords=[sta], dims=['sta'])
lon = xr.DataArray(lon, coords=[sta], dims=['sta'])

tm_sta = tm_da.interp(lon=lon,lat=lat, method='linear')
print(tm_sta)

(59,) (38,) (38, 59)
(824,) (824,) (824,)
<xarray.DataArray (sta: 824)>
array([ 3.52374508,  3.65335251,  3.87740995,  3.9520732 ,  3.74844743,
        5.05795247,  4.75724314,  4.49075102,  4.159793  ,  5.81821657,
        5.60242073,  5.01139382,  4.80508482,  4.43230736,  6.42337719,
        6.38833752,  5.46156473,  5.64573348,  4.92425931,  5.09554812,
        6.59591309,  5.84768579,  5.42006229,  5.68523452,  5.11411655,
        5.39373829,  4.72990406,  4.46968329,  6.6056651 ,  6.67644514,
        6.20319159,  5.38781119,  5.80669322,  5.08883373,  4.78851907,
        5.02036858,  4.65097341,  8.21748659,  6.71931713,  6.42942578,
        6.73127258,  6.47358966,  6.10215397,  5.73662455,  5.98905771,
        5.26029946,  5.65827441,  5.00126149,  5.05921697,  4.6664801 ,
        8.06956881,  8.15506147,  8.05243774,  8.0267781 ,  7.93440504,
        7.78544205,  8.41353516,  8.20428608,  7.65125324,  8.53499369,
        8.59017601,  8.41740702,  8.32061453,  7.51702019,  8.65

In [23]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker

fig = plt.figure(figsize=(12,8))
ax1 = fig.add_axes([0.1, 0.1, 0.8, 0.4],projection = ccrs.PlateCarree(central_longitude=90))
leftlon, rightlon, lowerlat, upperlat = (0,180,0,90)
img_extent = [leftlon, rightlon, lowerlat, upperlat]
ax1.set_extent(img_extent, crs=ccrs.PlateCarree())
# ax1.add_feature(cfeature.COASTLINE.with_scale('50m')) 
# ax1.add_feature(cfeature.LAKES, alpha=0.5)
ax1.set_xticks(np.arange(60,150+30,30), crs=ccrs.PlateCarree())
ax1.set_yticks(np.arange(0,60+30,30), crs=ccrs.PlateCarree())
lon_formatter = cticker.LongitudeFormatter()
lat_formatter = cticker.LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)
c1 = ax1.scatter(lon,lat,s=2, zorder=0,c=tm_sta,transform=ccrs.PlateCarree(), cmap=plt.cm.Reds)

position=fig.add_axes([0.35, 0.02,  0.35, 0.025])
fig.colorbar(c1,cax=position,orientation='horizontal',format='%d',)

<Figure size 864x576 with 2 Axes>

### 从网格到网格

1.利用cdo的remap函数  cdo remapbil,r360x180 infile outfile

2.xarray dataarray.interp

In [25]:
tm_da = xr.DataArray(tm_grid, coords=[lat_grid, lon_grid], dims=['lat', 'lon'])
# print(tm_da)

lon_grid25 = np.arange(75.0, 135.5, 2.5) 
lat_grid25 = np.arange(16.0, 55.5, 2.5) 

tm_grid25 = tm_da.interp(lon=lon_grid25,lat=lat_grid25, method='linear')

In [27]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker

fig = plt.figure(figsize=(12,8))
ax1 = fig.add_axes([0.1, 0.1, 0.8, 0.4],projection = ccrs.PlateCarree(central_longitude=90))
leftlon, rightlon, lowerlat, upperlat = (0,180,0,90)
img_extent = [leftlon, rightlon, lowerlat, upperlat]
ax1.set_extent(img_extent, crs=ccrs.PlateCarree())
# ax1.add_feature(cfeature.COASTLINE.with_scale('50m')) 
# ax1.add_feature(cfeature.LAKES, alpha=0.5)
ax1.set_xticks(np.arange(60,150+30,30), crs=ccrs.PlateCarree())
ax1.set_yticks(np.arange(0,60+30,30), crs=ccrs.PlateCarree())
lon_formatter = cticker.LongitudeFormatter()
lat_formatter = cticker.LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)
c1 = ax1.contourf(lon_grid25,lat_grid25,tm_grid25, zorder=0,#levels =np.arange(-10,10.5,0.5) , 
                     extend = 'both', transform=ccrs.PlateCarree(), cmap=plt.cm.Reds)

position=fig.add_axes([0.35, 0.02,  0.35, 0.025])
fig.colorbar(c1,cax=position,orientation='horizontal',format='%d',)

<Figure size 864x576 with 2 Axes>

## 时间插值

### 站点数据时间内插

当做序列处理，可利用pandas的resample函数或xarray的interp函数实现。

### 网格数据时间内插

同样可用xarray的interp函数实现。

In [64]:
import xarray as xr
import datetime as dt

f = xr.open_dataset('/home/mw/input/moyu1828/air.mon.mean.nc')
# print(f)
data = f.air.loc[np.array(['1979-01-01','1979-02-01','1979-04-01','1979-05-01'],dtype=np.datetime64),850]
# print(data)

data_interp = data.interp(time=np.array(['1979-01-01','1979-02-01','1979-03-01','1979-04-01','1979-05-01'],dtype=np.datetime64), method='linear')
print(data_interp)

<xarray.DataArray 'air' (time: 5, lat: 73, lon: 144)>
array([[[-24.08998871, -24.08998871, -24.08998871, ..., -24.08998871,
         -24.08998871, -24.08998871],
        [-24.29999542, -24.34999847, -24.38999176, ..., -24.15999603,
         -24.20999908, -24.26000214],
        [-24.5799942 , -24.59999847, -24.61998749, ..., -24.51999664,
         -24.54000092, -24.55998993],
        ...,
        [ -6.08998871,  -6.08998871,  -6.08998871, ...,  -6.10999298,
          -6.09999847,  -6.08998871],
        [ -6.94998932,  -6.94998932,  -6.93999481, ...,  -6.9799881 ,
          -6.96999359,  -6.95999908],
        [ -8.2299881 ,  -8.2299881 ,  -8.2299881 , ...,  -8.2299881 ,
          -8.2299881 ,  -8.2299881 ]],

       [[-27.96999359, -27.96999359, -27.96999359, ..., -27.96999359,
         -27.96999359, -27.96999359],
        [-27.30998993, -27.30998993, -27.29999542, ..., -27.37999725,
         -27.35999298, -27.3299942 ],
        [-26.65000153, -26.55998993, -26.49999237, ..., -27.0100021